# Genre Clustering

There are a lot of genres in the Spotify dataset (>5000). We'd like to see if the Spotify Audio features can be used to differentiate between different genres, but to do this we need to know that there is a distinct difference between the genres that we choose to compare. (For example, there is probably a distinct difference between the genres of Dubstep and Classical Piano, but probably less so between Rock and Alternative Rock).

To this end, we are going to attempt to cluster the genres into similar groups, and have a look at how homogenous the audio features are within some of the groups.

## Import Libraries

In [1]:
import pandas as pd
import numpy as np

import json
import time
import sys
import os
from collections import Counter, defaultdict

# import seaborn
# from matplotlib import pyplot as plt

# import sklearn
# from sklearn.cluster import KMeans, SpectralCoclustering, AgglomerativeClustering
# from sklearn.metrics import silhouette_score
# from sklearn.random_projection import GaussianRandomProjection, SparseRandomProjection
# from sklearn.manifold import TSNE
# import shared_functions as sf
# import clustergrammer2 as cg
# from clustergrammer2 import net
# from scipy.cluster.hierarchy import dendrogram, linkage

## Load Genre Co-Counts Data

In [35]:
if not os.path.isfile("genre_links.pkl"):
    data = json.load(open("../all_artist_info.json", "r"))
    artist_genres = []
    for artist_id in data:
        if artist_id == data[artist_id]["id"]:
            artist_genres.append((artist_id, data[artist_id]["genres"]))

    genre_links = {}
    for artist in artist_genres:
        genres = artist[1]
        if len(genres) >= 1:

            for g1 in genres:
                if g1 not in genre_links:
                    genre_links[g1] = {}

                for g2 in genres:
                    if g2 not in genre_links[g1]:
                        genre_links[g1][g2] = 0
                    genre_links[g1][g2] += 1

    df = pd.DataFrame(genre_links).fillna(0)
    df = df.astype(pd.SparseDtype("float", 0))
    df = df[list(df.index)]
    df.to_pickle("genre_links.pkl")

else:
    df = pd.read_pickle("genre_links.pkl")
    
df

,samurai trap,traprun,arab trap,boogie-woogie,piano blues,barrelhouse piano,jazz piano,stride,jump blues,blues,...,muzica bisericeasca,yunnan traditional,whale song,rakugo,school choir,mazandarani folk,massage,musica timor-leste,marcha funebre,chhattisgarhi pop
samurai trap,52.0,5.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
traprun,5.0,204.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
arab trap,1.0,6.0,48.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
boogie-woogie,0.0,0.0,0.0,127.0,46.0,20.0,4.0,10.0,10.0,23.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
piano blues,0.0,0.0,0.0,46.0,81.0,19.0,2.0,6.0,8.0,16.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
mazandarani folk,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0
massage,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0
musica timor-leste,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0
marcha funebre,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0


### Calculate Normed Genre Links

In [165]:
normed_genre_links = {}
for genre in genre_links:
    genre_count = genre_links[genre][genre]
    normed_genre_links[genre] = {}
    for g2 in genre_links[genre]:
        normed_genre_links[genre][g2] = genre_links[genre][g2] / genre_count
normed_genre_links

{'samurai trap': {'samurai trap': 1.0,
  'traprun': 0.09615384615384616,
  'arab trap': 0.019230769230769232},
 'boogie-woogie': {'boogie-woogie': 1.0,
  'piano blues': 0.36220472440944884,
  'barrelhouse piano': 0.15748031496062992,
  'jazz piano': 0.031496062992125984,
  'stride': 0.07874015748031496,
  'jump blues': 0.07874015748031496,
  'blues': 0.18110236220472442,
  'chicago blues': 0.05511811023622047,
  'electric blues': 0.11023622047244094,
  'harmonica blues': 0.023622047244094488,
  'louisiana blues': 0.06299212598425197,
  'memphis blues': 0.031496062992125984,
  'new orleans blues': 0.06299212598425197,
  'traditional blues': 0.12598425196850394,
  'jazz organ': 0.007874015748031496,
  'rock-and-roll': 0.023622047244094488,
  'soul blues': 0.023622047244094488,
  'texas blues': 0.023622047244094488,
  'big band': 0.03937007874015748,
  'new orleans jazz': 0.015748031496062992,
  'swing': 0.047244094488188976,
  'vintage jazz': 0.023622047244094488,
  'german blues': 0.023

### Calculate Jaccard Similarities

In [166]:
all_genres = df.columns
num_genres = len(all_genres)

genre_counts = {}
for genre in all_genres:
    genre_counts[genre] = df[genre][genre]

jaccard = {}
for g1 in genre_links:
    jaccard[g1] = {}
    for g2 in genre_links[g1]:
        overlap = genre_links[g1][g2]
        j_score = overlap / (genre_counts[g1] + genre_counts[g2] - overlap)
        jaccard[g1][g2] = j_score
jaccard

{'samurai trap': {'samurai trap': 1.0,
  'traprun': 0.0199203187250996,
  'arab trap': 0.010101010101010102},
 'boogie-woogie': {'boogie-woogie': 1.0,
  'piano blues': 0.2839506172839506,
  'barrelhouse piano': 0.091324200913242,
  'jazz piano': 0.015873015873015872,
  'stride': 0.03571428571428571,
  'jump blues': 0.05319148936170213,
  'blues': 0.07076923076923076,
  'chicago blues': 0.028,
  'electric blues': 0.03825136612021858,
  'harmonica blues': 0.011363636363636364,
  'louisiana blues': 0.04081632653061224,
  'memphis blues': 0.022727272727272728,
  'new orleans blues': 0.045454545454545456,
  'traditional blues': 0.056140350877192984,
  'jazz organ': 0.004464285714285714,
  'rock-and-roll': 0.007653061224489796,
  'soul blues': 0.013215859030837005,
  'texas blues': 0.014925373134328358,
  'big band': 0.01483679525222552,
  'new orleans jazz': 0.00749063670411985,
  'swing': 0.01764705882352941,
  'vintage jazz': 0.013513513513513514,
  'german blues': 0.01694915254237288,
  

In [169]:
j_matrix = pd.DataFrame(jaccard).fillna(0)[all_genres]
j_matrix

,samurai trap,traprun,arab trap,boogie-woogie,piano blues,barrelhouse piano,jazz piano,stride,jump blues,blues,...,muzica bisericeasca,yunnan traditional,whale song,rakugo,school choir,mazandarani folk,massage,musica timor-leste,marcha funebre,chhattisgarhi pop
samurai trap,1.000000,0.01992,0.010101,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
traprun,0.019920,1.00000,0.024390,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
arab trap,0.010101,0.02439,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
boogie-woogie,0.000000,0.00000,0.000000,1.000000,0.283951,0.091324,0.015873,0.035714,0.053191,0.070769,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
piano blues,0.000000,0.00000,0.000000,0.283951,1.000000,0.109195,0.009615,0.025210,0.055556,0.055944,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
mazandarani folk,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
massage,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
musica timor-leste,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
marcha funebre,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [171]:
for g in all_genres:
    print(g)

samurai trap
traprun
arab trap
boogie-woogie
piano blues
barrelhouse piano
jazz piano
stride
jump blues
blues
chicago blues
electric blues
harmonica blues
louisiana blues
memphis blues
new orleans blues
traditional blues
jazz organ
rock-and-roll
soul blues
texas blues
big band
new orleans jazz
swing
vintage jazz
german blues
jazz blues
rhythm and blues
bebop
cool jazz
acoustic blues
country blues
delta blues
modern blues
rockabilly
swamp blues
swing revival
harlem renaissance
ragtime
tin pan alley
honky-tonk piano
dixieland
southern soul
jug band
piedmont blues
synth funk
experimental ambient
funk
gangster rap
motown
new jack swing
p funk
popping
post-disco
quiet storm
post-disco soul
indie soul
new french touch
canadian contemporary r&b
future bass
austindie
indie jazz
wonky
experimental r&b
australian alternative pop
abstract beats
alternative hip hop
alternative r&b
indie r&b
vapor twitch
stomp pop
outsider
funk mexicano
future funk
miami indie
disco
soul
electro
funk rock
minneapol

synthwave
minimal synth
etherpop
singaporean pop
vapor soul
singaporean singer-songwriter
southeast asian post-rock
singaporean metal
singaporean hip hop
irish metal
irish black metal
folk metal
irish death metal
symphonic death metal
crossover thrash
new wave of thrash metal
speed metal
thrash metal
progressive sludge
atmospheric black metal
avant-garde black metal
cascadian black metal
post-black metal
voidgaze
belfast metal
dissonant death metal
folk black metal
viking metal
goregrind
antiviral pop
neo-trad doom metal
neo-trad metal
black metal
pagan black metal
progressive groove metal
cryptic black metal
icelandic black metal
technical black metal
atmospheric sludge
funeral doom
ambient fusion
zambian pop
greek metal
deep downtempo fusion
relaxative
future ambient
downtempo fusion
melodic thrash
progressive thrash
technical thrash
new england metal
texas hardcore
death metal
groove metal
black speed metal
new wave of speed metal
brazilian death metal
brazilian metal
brazilian thra

ska argentino
traditional ska
post-punk argentina
indie platense
cantautora argentina
hungarian edm
cubaton
cuban rumba
bachata dominicana
charanga
electro bailando
reggaeton chileno
dominican pop
reggaeton flow
trap latino
flamenco urbano
jazz colombiano
italian contemporary classical
danish contemporary classical
swedish classical
austrian contemporary classical
portuguese contemporary classical
spanish contemporary classical
czech contemporary classical
irish contemporary classical
theremin
swiss contemporary classical
mexican classical
thall
minneapolis metal
instrumental progressive metal
american grindcore
australian metalcore
cosmic death metal
uk post-metal
canadian metalcore
bulgarian metal
industrial black metal
downtempo deathcore
canadian post-hardcore
russian metalcore
progressive alternative
christian hardcore
florida death metal
indian metal
pennsylvania hardcore
swancore
st petersburg fl indie
kelowna bc indie
british alternative rock
australian post-hardcore
tampa indi

uk diy punk
scream rap
portsmouth indie
scottish americana
lithuanian hip hop
lithuanian rock
vaikiskos dainos
lithuanian folk
manila sound
kundiman
italian occult psychedelia
cardiff indie
techno rave
finnish techno
cymraeg
welsh choir
khaleeji iraqi
khaliji
yemeni traditional
jalsat
baluchi folk
kurdish rock
kurdish pop
kawaii metal
chinese minyao
kazakh traditional
chinese singer-songwriter
chinese metal
illbient
musica menorquina
galician jazz
beat poetry
hokkien pop
taiwanese pop
taiwan campus folk
folclore portugues
cancoes infantis
musica acoriana
fado instrumental
guitarra portuguesa
klezmer
brass ensemble
muzica populara
romanian folk
georgian electronic
cuban electronic
austrian techno
gamecore
musica sergipana
musica piauiense
organetto
musica cristiana guatemalteca
alberta hip hop
swedish folk pop
neue volksmusik
musikkorps
british brass band
tanzlmusi
necrogrind
polish metalcore
cumbia sonorense
cumbia salvadorena
musica istmena
tropical tecladista
liechtenstein
gamelan
is

In [176]:
for g in all_genres:
    if "k-" in g:
        print(g)

rock-and-roll
folk-pop
k-pop boy group
slack-key guitar
truck-driving country
k-rock
k-indie
k-pop
mexican rock-and-roll
musik anak-anak
swedish rock-and-roll
k-rap
classic k-pop
k-pop girl group


## Calculate/Load Jaccard Similarities

In [160]:
if not os.path.isfile("jaccard_matrix.pkl"):
    all_genres = df.columns
    num_genres = len(all_genres)

    # Construct list of Genre Counts for easier access
    genre_counts = {}
    for genre in all_genres:
        genre_counts[genre] = df[genre][genre]

    # Calculate Jaccard Similarities
    print("Calculating Jaccard Similarities:")
    jaccard = pd.DataFrame(np.identity(num_genres))
    jaccard.index = all_genres
    jaccard.columns = all_genres

    for i in range(num_genres):
        if i % 10 == 0:
            sys.stdout.write(f"\r{i} / {num_genres}")
            sys.stdout.flush()

        g_i = all_genres[i]
        g_i_count = genre_counts[g_i]

        for j in range(i+1, num_genres):
            g_j = all_genres[j]
            overlap = df[g_i][g_j]

            if overlap == 0:
                j_score = 0
            else:
                j_score = overlap / (g_i_count + genre_counts[g_j] - overlap)

            jaccard[g_i][g_j] = j_score
            jaccard[g_j][g_i] = j_score

    print("\rJaccard Similarity calculations complete.")
    print("Converting to Sparse Matrix...")
    jaccard = jaccard.astype(pd.SparseDtype("float", 0))
    print("Saving Jaccard Matrix...")
    jaccard.to_pickle("jaccard_matrix.pkl")
    
else:
    jaccard = pd.read_pickle("jaccard_matrix.pkl")
    
jaccard

Calculating Jaccard Similarities:
Jaccard Similarity calculations complete.
Converting to Sparse Matrix...
Saving Jaccard Matrix...


,samurai trap,traprun,arab trap,boogie-woogie,piano blues,barrelhouse piano,jazz piano,stride,jump blues,blues,...,muzica bisericeasca,yunnan traditional,whale song,rakugo,school choir,mazandarani folk,massage,musica timor-leste,marcha funebre,chhattisgarhi pop
samurai trap,1.000000,0.01992,0.010101,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
traprun,0.019920,1.00000,0.024390,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
arab trap,0.010101,0.02439,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
boogie-woogie,0.000000,0.00000,0.000000,1.000000,0.283951,0.091324,0.015873,0.035714,0.053191,0.070769,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
piano blues,0.000000,0.00000,0.000000,0.283951,1.000000,0.109195,0.009615,0.025210,0.055556,0.055944,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
mazandarani folk,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
massage,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
musica timor-leste,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
marcha funebre,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


## Check parameters of the Data

Have a quick look at some relevant stats about the data

### Sparsity
What percentage of the DataFrame is just zero-values?

In [159]:
print(f"Sparsity: {round(1 - df.sparse.density, 5)*100}%")

Sparsity: 99.529%


### Number of Connections

In [38]:
connection_list = (df != 0).sum()
print(f"Total number of genres: {len(connection_list)}")
print(f"Maximum number of connections: {connection_list.max()} ({connection_list[connection_list == connection_list.max()].index[0]})")
print(f"Minimum number of connections: {connection_list.min()} (>1 so every genre has at least 1 connection as well as itself)")
print(f"Median number of connections: {connection_list.median()}")
print(f"Mean number of connections: {connection_list.mean()}")

Total number of genres: 5577
Maximum number of connections: 315 (art pop)
Minimum number of connections: 1 (>1 so every genre has at least 1 connection as well as itself)
Median number of connections: 16.0
Mean number of connections: 26.25838264299803


## Explore the Genres

In [198]:
word_dict = {}
ngram_dict = {}

for genre in genre_links:
    words = genre.split(" ")
    
    for word in words:
        if word not in word_dict:
            word_dict[word] = 0
        word_dict[word] += 1
        
    if len(words) >= 2:
        for i in range(len(words)):
            for j in range(i+1, len(words)):
                w1 = words[i]
                w2 = words[j]
                ngram = " ".join(sorted([w1, w2]))
                if ngram not in ngram_dict:
                    ngram_dict[ngram] = 0
                ngram_dict[ngram] += 1
    
sorted(word_dict.items(), key=lambda x: x[1], reverse=True)

[('indie', 485),
 ('pop', 411),
 ('metal', 344),
 ('rock', 332),
 ('hop', 186),
 ('hip', 184),
 ('rap', 170),
 ('folk', 168),
 ('jazz', 136),
 ('classical', 134),
 ('punk', 132),
 ('deep', 110),
 ('musica', 108),
 ('house', 85),
 ('classic', 82),
 ('japanese', 75),
 ('german', 71),
 ('italian', 69),
 ('black', 69),
 ('traditional', 66),
 ('experimental', 65),
 ('trap', 64),
 ('russian', 64),
 ('swedish', 63),
 ('electronic', 62),
 ('alternative', 59),
 ('hardcore', 54),
 ('techno', 52),
 ('new', 52),
 ('modern', 51),
 ('death', 50),
 ('canadian', 49),
 ('blues', 49),
 ('australian', 46),
 ('piano', 46),
 ('brazilian', 46),
 ('reggae', 44),
 ('finnish', 44),
 ('contemporary', 43),
 ('french', 42),
 ('polish', 40),
 ('funk', 39),
 ('indonesian', 39),
 ('progressive', 39),
 ('country', 39),
 ('spanish', 39),
 ('dutch', 38),
 ('gospel', 37),
 ('uk', 34),
 ('irish', 34),
 ('chinese', 34),
 ('instrumental', 33),
 ('norwegian', 33),
 ('soul', 33),
 ('music', 31),
 ('latin', 30),
 ('singer-son

In [199]:
sorted(ngram_dict.items(), key=lambda x: x[1], reverse=True)

[('hip hop', 182),
 ('black metal', 62),
 ('death metal', 45),
 ('classic pop', 43),
 ('indie rock', 33),
 ('classical piano', 32),
 ('classical contemporary', 27),
 ('alternative rock', 21),
 ('indie pop', 17),
 ('african south', 17),
 ('deep house', 16),
 ('rap underground', 16),
 ('pop punk', 16),
 ('metal power', 15),
 ('new wave', 14),
 ('rock stoner', 14),
 ('folk indie', 13),
 ('house tech', 13),
 ('jazz modern', 13),
 ('classic rock', 12),
 ('metal progressive', 12),
 ('hard rock', 11),
 ('old school', 11),
 ('folk metal', 11),
 ('psychedelic rock', 11),
 ('metal thrash', 10),
 ('deep rock', 10),
 ('garage rock', 10),
 ('folk rock', 9),
 ('modern rock', 9),
 ('melodic metal', 9),
 ('pop rock', 9),
 ('pop swedish', 9),
 ('brazilian metal', 9),
 ('italian metal', 9),
 ('punk rock', 9),
 ('italian pop', 8),
 ('classical performance', 8),
 ('rock swedish', 8),
 ('indonesian pop', 8),
 ('pop rap', 8),
 ('heavy metal', 8),
 ('metal symphonic', 8),
 ('rock spanish', 8),
 ('doom metal'

## Manual Clustering
The Co-occurrence matrix is very sparse, which seems to imply that not all genres are actually connected with each other. Can we form clusters by just grouping together any genres that are linked at all, or will that devolve into one large cluster just through a few outliers being linked with everything?

In [39]:
a = set([1, 2, 3])
n = set([3, 4, 5])
d = set([5, 6, 7])
print(set.union(*[a, n, d],[8, 9]))

{1, 2, 3, 4, 5, 6, 7, 8, 9}


In [186]:
def create_genre_groups(gmap):
    groups = []
    
    for base_genre in gmap:
        r_genres = set(gmap[base_genre].keys())
        matches = []
        
        for i in range(len(groups)):
            # If the group has any common elements, group them together
            if not (r_genres.isdisjoint(groups[i])):
                groups[i] = groups[i].union(r_genres)
                matches.append(i)
        
        if len(matches) > 0:
            untouched_groups = [groups[i] for i in range(len(groups)) if i not in matches]
            merged_group = set.union(*[groups[i] for i in matches], r_genres)
            groups = untouched_groups + [merged_group]
        
        else:
            # If no groups with common elements were found, create a new group
            groups.append(r_genres)
        
    return groups

output = create_genre_groups(normed_genre_links)

In [215]:
print(list(map(len, create_genre_groups(normed_genre_links))))

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 5523]


### Testing Co-occurrence Threshold

Okay, that didn't really work. We've ended up splitting off a whole 5 genres, and are left with a central supercluster of 5523 genres. It looks like the genres are generally too interconnected. But maybe we can start cutting some of those connections to try and break this down.

What happens if we apply a threshold (e.g. for two genres to count as "related" their co-occurrence has to be above a certain value)?

#### With Normed Genre Links

In [200]:
total_links = 0
link_count = {i: 0 for i in range(10)}
for g in normed_genre_links:
    for rg, val in normed_genre_links[g].items():
        # Skip comparing to itself
        if (g == rg) or (val == 0):
            continue
        total_links += 1
        i = 0
        while (i+1)/10 < normed_genre_links[g][rg]:
            i += 1
        link_count[i] += 1
        
for n in link_count:
    link_count[n] = round(link_count[n] / total_links * 100, 2)
    
for val, perc in link_count.items():
    print(f"{val/10} < C <= {(val+1)/10}: {perc}%")

0.0 < C <= 0.1: 91.62%
0.1 < C <= 0.2: 5.05%
0.2 < C <= 0.3: 1.71%
0.3 < C <= 0.4: 0.82%
0.4 < C <= 0.5: 0.42%
0.5 < C <= 0.6: 0.21%
0.6 < C <= 0.7: 0.11%
0.7 < C <= 0.8: 0.05%
0.8 < C <= 0.9: 0.02%
0.9 < C <= 1.0: 0.0%


#### With Jaccard Similarity

In [189]:
total_links = 0
link_count = {i: 0 for i in range(10)}
for g in jaccard:
    for rg, val in jaccard[g].items():
        # Skip comparing to itself
        if (g == rg) or (val == 0):
            continue
        total_links += 1
        i = 0
        while (i+1)/10 < jaccard[g][rg]:
            i += 1
        link_count[i] += 1
        
for n in link_count:
    link_count[n] = round(link_count[n] / total_links * 100, 2)
    
for val, perc in link_count.items():
    print(f"{val/10} < C <= {(val+1)/10}: {perc}%")

0.0 < C <= 0.1: 97.12%
0.1 < C <= 0.2: 2.05%
0.2 < C <= 0.3: 0.52%
0.3 < C <= 0.4: 0.2%
0.4 < C <= 0.5: 0.05%
0.5 < C <= 0.6: 0.04%
0.6 < C <= 0.7: 0.01%
0.7 < C <= 0.8: 0.0%
0.8 < C <= 0.9: 0.0%
0.9 < C <= 1.0: 0.0%


Okay, this looks more promising. With a large percentage of co-occurrences already at very low rates, we should be able to eliminate most of them by applying a low threshold that doesn't remove too much of the existing clusters.

### Checking Threshold Group Sizes

In [225]:
def apply_threshold(gmap, threshold):
    trimmed_gmap = {}
    for genre in gmap:
        trimmed_gmap[genre] = {}
        for related_genre in gmap[genre]:
            if gmap[genre][related_genre] >= threshold:
                trimmed_gmap[genre][related_genre] = gmap[genre][related_genre]
    return trimmed_gmap

#### Normed Genre Links

In [217]:
for t in range(1, 10):
    threshold = t/10
    print(threshold)
    trimmed_genre_links = apply_threshold(normed_genre_links, threshold)
    genre_groups = create_genre_groups(trimmed_genre_links)
    group_sizes = [len(g) for g in genre_groups]
    print(sorted(Counter(group_sizes).items()))
    print()

0.1
[(1, 1285), (2, 132), (3, 48), (4, 22), (5, 11), (6, 5), (7, 7), (8, 3), (9, 1), (10, 1), (11, 3), (12, 2), (13, 1), (14, 5), (15, 2), (21, 2), (33, 1), (43, 1), (58, 1), (3273, 1)]

0.2
[(1, 2687), (2, 268), (3, 85), (4, 45), (5, 14), (6, 17), (7, 13), (8, 14), (9, 4), (10, 6), (11, 1), (12, 4), (13, 3), (14, 2), (16, 1), (17, 3), (18, 2), (19, 2), (20, 1), (23, 1), (25, 1), (27, 1), (28, 1), (30, 1), (37, 1), (49, 1), (70, 1), (127, 1), (745, 1)]

0.3
[(1, 3722), (2, 297), (3, 80), (4, 44), (5, 20), (6, 14), (7, 6), (8, 8), (9, 4), (10, 2), (11, 1), (12, 3), (13, 2), (15, 2), (18, 2), (21, 1), (24, 1), (25, 2), (26, 1), (27, 1), (212, 1)]

0.4
[(1, 4449), (2, 250), (3, 67), (4, 27), (5, 8), (6, 9), (7, 4), (8, 3), (9, 2), (10, 2), (11, 1), (13, 2), (15, 1), (16, 1), (17, 1), (50, 1)]

0.5
[(1, 4939), (2, 170), (3, 35), (4, 14), (5, 5), (6, 4), (7, 3), (8, 2), (10, 1), (13, 2), (15, 1)]

0.6
[(1, 5272), (2, 89), (3, 13), (4, 11), (5, 3), (6, 2), (8, 1), (9, 1)]

0.7
[(1, 5424), (2

In [224]:
trimmed_genre_links = apply_threshold(normed_genre_links, 0.7)
genre_groups = create_genre_groups(trimmed_genre_links)
for g in genre_groups:
    if len(g) >= 3:
        print(g)
        print()

{'hard bop', 'bebop', 'cool jazz'}

{'new romantic', 'new wave', 'synthpop', 'new wave pop'}

{'dirty south rap', 'crunk', 'gangster rap', 'southern hip hop'}

{'sierreno', 'nuevo regional mexicano', 'corrido'}

{'urban contemporary', 'r&b', 'new jack swing'}

{'blues', 'traditional blues', 'acoustic blues'}

{'christian music', 'worship', 'christian alternative rock'}

{'east coast hip hop', 'hardcore hip hop', 'hip hop'}

{'hindustani classical', 'hindustani vocal', 'carnatic vocal', 'indian classical', 'hindustani instrumental', 'carnatic instrumental', 'carnatic'}

{'mande pop', 'malian blues', 'world', 'afropop'}

{'swedish hip hop', 'swedish gangsta rap', 'swedish trap', 'swedish trap pop'}

{'alternative metal', 'post-grunge', 'rap metal', 'nu metal'}



#### Jaccard Similarities

In [219]:
for t in range(1, 10):
    threshold = t/10
    print(threshold)
    trimmed_genre_links = apply_threshold(jaccard, threshold)
    genre_groups = create_genre_groups(trimmed_genre_links)
    group_sizes = [len(g) for g in genre_groups]
    print(sorted(Counter(group_sizes).items()))
    print()

0.1
[(1, 3599), (2, 298), (3, 93), (4, 41), (5, 25), (6, 11), (7, 6), (8, 3), (9, 2), (10, 2), (11, 3), (13, 4), (14, 2), (15, 1), (17, 2), (23, 1), (40, 1), (419, 1)]

0.2
[(1, 4839), (2, 179), (3, 34), (4, 20), (5, 4), (6, 5), (7, 3), (8, 3), (9, 2), (11, 3), (12, 1), (13, 1), (27, 1)]

0.3
[(1, 5280), (2, 81), (3, 18), (4, 8), (5, 2), (6, 2), (8, 1), (9, 1), (10, 1)]

0.4
[(1, 5462), (2, 44), (3, 5), (4, 3)]

0.5
[(1, 5520), (2, 22), (3, 3), (4, 1)]

0.6
[(1, 5567), (2, 5)]

0.7
[(1, 5575), (2, 1)]

0.8
[(1, 5577)]

0.9
[(1, 5577)]



In [226]:
trimmed_genre_links = apply_threshold(jaccard, 0.3)
genre_groups = create_genre_groups(trimmed_genre_links)
for g in genre_groups:
    if len(g) > 3:
        print(g)
        print()

{'hard bop', 'bebop', 'cool jazz', 'soul jazz', 'jazz'}

{'uplifting trance', 'trance', 'progressive trance', 'progressive house'}

{'new romantic', 'new wave', 'synthpop', 'new wave pop'}

{'contemporary country', 'country road', 'modern country rock', 'country'}

{'pop dance', 'tropical house', 'edm', 'electro house'}

{'sleaze rock', 'hard rock', 'classic rock', 'soft rock', 'glam metal', 'folk rock', 'mellow gold', 'album rock'}

{'blues', 'electric blues', 'modern blues', 'traditional blues'}

{'christian alternative rock', 'christian music', 'ccm', 'worship'}

{'rap', 'hardcore hip hop', 'east coast hip hop', 'trap', 'dirty south rap', 'pop rap', 'hip hop', 'gangster rap', 'southern hip hop'}

{'latin alternative', 'latin rock', 'mexican rock', 'rock en espanol'}

{'classic iskelma', 'finnish pop', 'finnish dance pop', 'classic finnish pop', 'iskelma', 'finnish hip hop'}

{'indian classical', 'hindustani classical', 'carnatic', 'carnatic vocal'}

{'latin pop', 'latin', 'latin hip

It looks like Jaccard might not be the best method for this process. By being a symmetrical linking method, it removes the possibility of very small sub-genres being linked to very large core-genres. E.g. If there was a subgenre G1 of the core-genre G2, and it only exclusively occurred with G2, then it would be safe to assume that we could include it into G2's cluster. However, the Jaccard Similarity would be 

> Len(G1 ∩ G2) / ( Len(G1) + Len(G2) - Len(G1 ∩ G2) )

We can see that if we took this to the extreme limits where `len(G1) ~= 0`, and `len(G2) ~= inf.`, even when `len(G1.intersection(G2)) = len(G1)` we still have a Jaccard Similarity of ~0.

Therefore, going forward, it looks like the asymmetric _Normed Genre Links_ method will work better, as it allows small genres to have large links to big genres, even if the big genres don't have large links to those small genres.

## Threshold Clustering Class

In [212]:
class GroupClassifier():
    # For subsequently lower thresholds, attempt to have a user-input name for each group.
    # Remember group names for future groups, and summarise results using them.
    def __init__(self, genre_links, threshold_start=90, threshold_cap=10, threshold_step=10, group_min_size=5):
        self.genre_links = genre_links
        self.threshold_start = threshold_start
        self.threshold_cap = threshold_cap
        self.threshold_step = threshold_step
        self.group_min_size = group_min_size
        self.named_groups = []
        self.threshold_groups = {}
        self.main()
        
    
    def main(self):
        self.get_genre_counts()
        self.get_normed_genre_links()
        self.get_jaccard_similarities()
        self.iterate_clustering()
    
    
    def get_genre_counts(self):
        genre_counts = {}
        for genre in self.genre_links:
            genre_counts[genre] = self.genre_links[genre][genre]
        self.genre_counts = genre_counts
    
    
    def get_normed_genre_links(self):
        genre_links = self.genre_links
        normed_genre_links = {}
        for g1 in self.genre_links:
            genre_count = self.genre_links[g1][g1]
            normed_genre_links[g1] = {}
            for g2 in genre_links[g1]:
                normed_genre_links[g1][g2] = genre_links[g1][g2] / genre_count
        self.normed_genre_links = normed_genre_links
        
    
    def get_jaccard_similarities(self):
        genre_links = self.genre_links
        genre_counts = self.genre_counts
        
        jaccard = {}
        for g1 in genre_links:
            jaccard[g1] = {}
            for g2 in genre_links[g1]:
                overlap = genre_links[g1][g2]
                j_score = overlap / (genre_counts[g1] + genre_counts[g2] - overlap)
                jaccard[g1][g2] = j_score
        
        self.jaccard = jaccard
    
    
    ### Clustering ##################################
    
    def iterate_clustering(self):
        t_start = self.threshold_start
        t_cap = self.threshold_cap - 1
        t_step = -1 * self.threshold_step
        print("Iterating")
        print(t_start, t_cap, t_step)
        for threshold in range(t_start, t_cap, t_step):
            print(threshold)
            
    
    
    def get_groups(self, threshold):
        trimmed_gmap = apply_threshold(self.gmap, threshold)
        genre_groups = create_genre_groups(trimmed_gmap)
        return genre_groups
        
        
    def get_full_groups(self):
        full_group_list = []
        for fg in self.named_groups:
            full_group = list(fg)
            while True:
                for index in range(len(full_group)):
                    element = full_group[index]
                    if type(element) == "int":
                        del full_group[index]
                        full_group += self.named_groups[index]
                        continue
                        
                # If we get here, there are no more numbers left
                # I have checked that there are no genre names that can be interpreted as numbers
                break
                
            full_group_list.append(full_group)
            
        return full_group_list
        
    
    def check_clusters(self, threshold):
        current_group_list = [g for g in self.get_groups(threshold) if len(g) >= self.group_min_size]
        
        full_group_list = self.get_full_groups()
        new_groups = []
        
        for current_group in current_group_list:
            group = current_group.copy()
            subgroups = []
            
            overlap_found = False
            while True:
                overlap_groups = [(n, g) for n, g in enumerate(full_group_list) if len(group.intersection(g)) > 0]
                
                if len(overlap_groups) == 0:
                    # Current group is not comprised of any subgroups
                    break
                
                max_group_index, max_group = max(overlap_groups, key=lambda x: len(x[1]))
                
                subgroups.append(max_group_index)
                group -= set(max_group)
                overlap_found = True
            
            # There are now 4 scenarios:
            # 1) If len(subgroups) == 0, it's a new cluster
            # 2) If len(subgroups) >= 2, it's a merging of clusters
            # 3) If len(subgroups) == 1, and len(group) > 0, it is an extension of an exisiting cluster
            # 4) If len(subgroups) == 1, and len(group) == 0, it IS an existing cluster.
            # Secnario 4 is the only scenario where we don't save the cluster.
            
            if (len(subgroups) == 1) and (len(group) == 0):
                # Scenario 4
                continue
                
            else:
                # Scenarios 1-3
                # Add subgroups to cluster
                group.update(subgroups)
                self.named_groups.append(group)
                new_groups.append(group)
                
        self.threshold_groups[threshold] = new_groups
        return new_groups
    
    
    def unfold_group(self, group):
        for i in range(len(group)):
            element = group[i]
            if type(element) == type(1):
                group[i] = self.unfold_group_by_index(element)
        return group
        
        
    def unfold_group_by_index(self, group_index):
        group = list(self.named_groups[group_index].copy())
        return self.unfold_group(group)
    
    
    def get_threshold_groups(self, index):
        key = self.threshold_groups.keys()[index]
        for i in range(index, -1, -1):
            # For each group in current threshold:
            #     Unfold group
            #     Check if the group has any overlap with existing groups in the current list
            #     If not, add to the current list
            pass
        
    def flatten(self, group):
        output = []
        for e in arr:
            if isinstance(e, list) or isinstance(e, set):
                print(e)
                output += flatt(e)
            else:
                output.append(e)
        return output
        
        
        
        
GC = GroupClassifier(genre_links)

Iterating
90 9 -10
90
80
70
60
50
40
30
20
10


In [ ]:
GC.threshold_groups[0.5000000000000001]

In [ ]:
len(GC.named_groups)

In [ ]:
GC.named_groups[8]

In [ ]:
print(GC.unfold_group(154))

In [ ]:
print("Current Groups:")
for g in current_group_list:
    print(g)
print("===\n")

full_group_list = []
for full_group in all_group_list:
    while True:
        for index in range(len(full_group)):
            element = full_group[index]
            if type(element) == "int":
                del full_group[index]
                full_group += all_group_list[index]
                continue
        # If we get here, there are no more numbers left
        # We have checked that no genre names can be interpreted as numbers
        break
    full_group_list.append(full_group)

print("Existing Groups:")
for g in full_group_list:
    print(g)
print("===\n")
    
for current_group in current_group_list:
    print("Original group:")
    print(current_group)
    group = current_group.copy()
    subgroups = []
    
    overlap_found = False
    while True:
        overlap_groups = [(n, g) for n, g in enumerate(full_group_list) if len(group.intersection(g)) > 0]
        if len(overlap_groups) == 0:
            print("no overlap groups")
            break
        max_group_index, max_group = max(overlap_groups, key=lambda x: len(x[1]))
        print(f"mgi: {max_group_index}, mg: {max_group}")
        if len(max_group) == 1:
            break
        subgroups.append(max_group_index)
        print(max_group)
        group -= set(max_group)
        overlap_found = True
    
    if overlap_found:  
        print("Subgroups:")
        print(subgroups)
        print("Leftovers:")
        print(group)
        print("Final outcome:")
        group.update(subgroups)
        print(group)
    print()

In [ ]:
for i in range(9, 1, -1):
    print(i)

In [ ]:
a = [[1,2,3,4,[9,10,[11,12]]],[5,6,7,8,{1,2,3}]]
a

In [ ]:
def flatt(arr):
    output = []
    for e in arr:
        if isinstance(e, list) or isinstance(e, set):
            print(e)
            output += flatt(e)
        else:
            output.append(e)
    return output

In [ ]:
flatt(a)

---

## Automated Spectral Clustering

In [ ]:
from sklearn.cluster import SpectralClustering

In [ ]:
clustering = SpectralClustering(n_clusters=10, n_components=300, assign_labels="discretize",
                                random_state=0, affinity="precomputed")
clustering.fit(df.values)

In [ ]:
count = {}
for i in clustering.labels_:
    if i not in count:
        count[i] = 0
    count[i] += 1
group_sizes = {}
for i in count:
    if count[i] not in group_sizes:
        group_sizes[count[i]] = 0
    group_sizes[count[i]] += 1
print(sorted(group_sizes.items(), key=lambda x: x[0], reverse=True))
print(count)
print(len(count))

In [ ]:
for k in range(1, 50):
    for i in range(len(clustering.labels_)):
        if clustering.labels_[i] == k:
            print(df.index[i])
    print()

In [ ]:
i = 2000
j = 50
clustering.labels_[i:i+j]

In [ ]:
clustering.labels_[2002]

In [ ]:
df.index[2002]